# INFO
- experiment: growing gooey
- stimulation: rsvp 100ms(stim) / 75ms(blank) / 2500ms break between tokens / 15 flashes per char / 6 tokens max per shape for a max time of 18 seconds.

- users tested: 1
- devices tested : 
    - OpenBCI: freq 125Hz / channels FC3,FCz,FC4,T7,C3,Cz,C4,T8,P7,P3,Pz,P4,P8,O1,O2,Oz

This code demonstrates how to read epoched data which already has been processed and used for predictions. It loads and concatenates each epoch to plot the data and visualize discriminated signals in correlation with stimulus.

# DEPENDENCIES

In [1]:
# BUILT-IN
import os
import math
from collections import OrderedDict

# DATAFRAMES
import numpy as np
import pandas as pd

# PLOTTING
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# MNE
from mne import concatenate_epochs, read_epochs

# SETTINGS

In [2]:
the_folder_path = "../data/growing_gooey" # relative datasets path
the_user = "compmonks" # check available users in data folder or add new ones
the_device = "openbci_v207" # available devices
the_units = "uVolts" # "uVolts" # "Volts" # unit of received data from device
the_montage = "standard_1020" # "standard_1020" (OpenBCI) # channels montage
the_markers = {'Non-Target': 2, 'Target': 1} # markers from stim data
conditions = OrderedDict()
conditions['2: Non-target'] = [the_markers["Non-Target"]]
conditions['1: Target'] = [the_markers["Target"]]
sns.set_context('talk') # seaborn settings
sns.set_style('white') # seaborn settings
diverging_color_palette = "coolwarm" # seaborn settings

# training data for comparison
training_folder_path = "../data/p300_speller/"
the_data_path = os.path.join(the_folder_path,the_device,the_user,"testing")

# UTILS

In [3]:
def plotConditions(the_epochs, unit, channels_num, diff_waveform ,conditions, palette,no_legend,scaling):
    """ Plot ERP conditions. """    
    
    confidence_interval = 97.5 # confidence interval in range [0, 100]
    bootstrap_samples = 1000 # number of bootstrap samples
    title = 'ERP Conditions'
    ylim = (-2,2) #(tuple): (ymin, ymax)
    if isinstance(conditions, dict):
        conditions = OrderedDict(conditions)
        palette = sns.color_palette(palette, len(conditions))
        if unit == "uVolts"and scaling:
            X = the_epochs.get_data() * 1e6 # convert from Volts to uVolts
        else:
            X = the_epochs.get_data()
        y = pd.Series(the_epochs.events[:, -1])
        times = the_epochs.times
        # get the amount of rows and columns and set axes
        fig_cell = math.ceil(math.sqrt(channels_num))
        fig, axes = plt.subplots(fig_cell, fig_cell, figsize=[12, 12],sharex=True, sharey=True)
        axes = [axes[row, col] for row in range(fig_cell) for col in range(fig_cell)]
        # make the subplots
        for ch in range(channels_num): 
            for cond, color in zip(conditions.values(), palette):
                sns.tsplot(np.average(X[y.isin(cond), ch],axis=0),
                           time = times, 
                           color = color,
                           n_boot = bootstrap_samples, 
                           ci = confidence_interval, 
                           ax = axes[int(ch)]
                          )
            if diff_waveform:
                diff = (np.nanmean(X[y == diff_waveform[1], ch], axis=0) - np.nanmean(X[y == diff_waveform[0], ch], axis=0))
                axes[ch].plot(times, diff, color='darkblue', lw=0.25)
            axes[ch].set_title(the_epochs.ch_names[ch])
            axes[ch].title.set_fontsize(8)
            axes[ch].set_ylim(ylim)
            axes[ch].axvline(x=0, ymin=ylim[0], ymax=ylim[1], color='grey',lw=0.5, label='_nolegend_')
            axes[ch].axvline(x=0.3, ymin=ylim[0], ymax=ylim[1], color='black',lw=0.5,ls=":", label='0.3')
        axes[0].set_ylabel('Amplitude (uV)')
        axes[-1].set_xlabel('Time (s)')
        if not no_legend:
            if diff_waveform:
                legend = (['{} - {}'.format(diff_waveform[1], diff_waveform[0])] + list(conditions.keys()))
            elif legend is None:
                legend = (list(conditions.keys()))
            else:
                legend += conditions.keys()        
            axes[-1].legend(legend,loc='best',fontsize='xx-small')
        sns.despine()
        plt.tight_layout()
        if title:
            fig.suptitle(title, fontsize=20)
    else:
        print("bad conditions")

# CONCATENATE EPOCHS AND PLOT CONDITIONS

In [ ]:
# testing data
raw = []
file_list = os.listdir(the_data_path)
for _f in file_list:
    if ".fif" in _f:
        read_fif = read_epochs(os.path.join(the_data_path,_f),preload=True, verbose=False)
        read_fif.events_id = the_markers
        read_fif.info['montage'] = the_montage
        read_fif.drop_bad(reject = {'eeg': 75e-6})
        if len(read_fif.get_data()) >0:
            raw.append(read_fif)
the_epochs = concatenate_epochs(raw)
# plot conditions
# 
plotConditions(
              the_epochs,
              the_units,
              16,
              diff_waveform = (2,1),
              conditions = conditions,
              palette = diverging_color_palette,
              no_legend = True,
              scaling=True)
#
plt.savefig(os.path.join(the_data_path,"CONDITIONS.png"),dpi = 800,format = "png")
plt.show()